In [1]:
from math import *
from expressions import *
from abc import ABC, abstractmethod

In [2]:
def f(x):
    return (3*x**2+x)*sin(x)

In [3]:
class Power():
    def __init__(self,base,exponent):
        self.base=base
        self.exponent=exponent

In [4]:
class Number():
    def __init__(self,number):
        self.number=number

In [5]:
class Variable():
    def __init__(self,symbol):
        self.symbol=symbol

In [6]:
Power(Variable("x"),Number(2))

In [7]:
class Product():
    def __init__(self,exp1,exp2):
        self.exp1=exp1
        self.exp2=exp2

In [8]:
Product(Number(3),Power(Variable("x"),Number(2)))

In [9]:
class Sum():
    def __init__(self,*exps):
        self.exps=exps

In [10]:
class Function():
    def __init__(self,name):
        self.name=name

In [11]:
class Apply():
    def __init__(self,function,argument):
        self.function=function
        self.argument=argument

In [12]:
f_expression=Product(
    Sum(
        Product(
            Number(3),
            Power(
                Variable("x"),
                Number(2))),
        Variable("x")),
    Apply(
        Function("sin"),
        Variable("x")))

In [13]:
Apply(Function("cos"),Sum(Power(Variable("x"),Number("3")),Number(-5)))

In [14]:
def f(y,z):
    return log(y**z)

In [15]:
Apply(Function('ln'),Power(Variable('y'),Variable('z')))

In [16]:
class Quotient():
    def __init__(self,numerator,denominator):
        self.numerator=numerator
        self.denominator=denominator

In [17]:
Quotient(Sum(Variable('a'),Variable('b')),Number(2))

In [18]:
class Difference():
    def __init__(self,exp1,exp2):
        self.exp1=exp1
        self.exp2=exp2

In [19]:
Difference(
    Power(Variable("b"),Number(2)),
    Product(Number(4),Product(Variable("a"),Variable("c")))
)

In [20]:
class Negative():
    def __init__(self,exp):
        self.exp=exp

In [21]:
Negative(Sum(Power(Variable('x'),Number(2)),Variable("y")))

In [22]:
A=Variable('a')
B=Variable('b')
C=Variable('c')
sqrt=Function('sqrt')

In [23]:
Quotient(
    Sum(
        Negative(B),
        Apply(
            sqrt,
            Difference(
                Power(B,Number(2)),
                Product(Number(4),Product(A,C))))),
        Product(Number(2),A))

In [24]:
def f(x):
    return (3*x**2+x)*sin(x)

In [25]:
def distinct_variables(exp):
    if isinstance(exp,Variable):
        return set(exp.symbol)
    elif isinstance(exp,Number):
        return set()
    elif isinstance(exp,Sum):
        return set().union(*[distinct_variables(exp) for exp in exp.exps])
    elif isinstance(exp,Product):
        return distinct_variables(exp.exp1).union(distinct_variables(exp.exp2))
    elif isinstance(exp,Power):
        return distinct_variables(exp.base).union(distinct_variables(exp.exponent))
    elif isinstance(exp,Apply):
        return distinct_variables(exp.argument)
    else:
        raise TypeError("Not a vaild expression")


In [26]:
distinct_variables(Variable('z'))

{'z'}

In [27]:
distinct_variables(Number(3))

set()

In [28]:
distinct_variables(f_expression)

{'x'}

In [46]:
class Expression(ABC):
    @abstractmethod
    def evaluate(self, **bindings):
        pass

In [47]:
class Number(Expression):
    def __init__(self,number):
        self.number=number
    def evaluate(self,**bindings):
        return self.number

In [31]:
Number(7).evaluate(x=3,y=6,q=-15)

7

In [32]:
class Variable(Expression):
    def __init__(self,symbol):
        self.symbol=symbol
    def evaluate(self,**bindings):
        try:
            return bindings[self.symbol]
        except:
            return KeyError("Variable '{}' is not bound.".format(self.symbol))

In [33]:
Variable("x").evaluate(x=5)

5

In [44]:
# # KeyError: "Variable 'x' is not bound." 
# Variable("x").evaluate(y=5)

In [34]:
class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) * self.exp2.evaluate(**bindings)

In [35]:
Product(Variable("x"),Variable("y")).evaluate(x=2,y=5)

10

In [36]:
_function_bindings={
    "sin":math.sin,
    "cos":math.cos,
    "ln":math.log
}
class Apply(Expression):
    def __init__(self,function,argument):
        self.function=function
        self.argument=argument
    def evaluate(self,**bindings):
        return _function_bindings[self.function.name](self.argument.evaluate(**bindings))

In [39]:
class Sum(Expression):
    def __init__(self, *exps):
        self.exps = exps
    def evaluate(self, **bindings):
        return sum([exp.evaluate(**bindings) for exp in self.exps])
    
class Power(Expression):
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent
    def evaluate(self, **bindings):
        return self.base.evaluate(**bindings) ** self.exponent.evaluate(**bindings)
    
class Difference(Expression):
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) - self.exp2.evaluate(**bindings)
    
class Quotient(Expression):
    def __init__(self,numerator,denominator):
        self.numerator = numerator
        self.denominator = denominator
    def evaluate(self, **bindings):
        return self.numerator.evaluate(**bindings) / self.denominator.evaluate(**bindings)

In [40]:
f_expression.evaluate(x=5)

-76.71394197305108

In [42]:
f(5)

-76.71394197305108

In [53]:
class Expression(ABC):
    @abstractmethod
    def evaluate(self, **bindings):
        pass
    def expand(self):
        pass

In [49]:
class Number(Expression):
    def __init__(self,number):
        self.number=number
    def evaluate(self,**bindings):
        return self.number
    def expand(self):
        return self

In [50]:
class Sum(Expression):
    def __init__(self, *exps):
        self.exps = exps
    def evaluate(self, **bindings):
        return sum([exp.evaluate(**bindings) for exp in self.exps])
    def expand(self):
        return Sum(*[exp.expand() for exp in exp.exps])

In [51]:
_function_bindings={
    "sin":math.sin,
    "cos":math.cos,
    "ln":math.log
}
class Apply(Expression):
    def __init__(self,function,argument):
        self.function=function
        self.argument=argument
    def evaluate(self,**bindings):
        return _function_bindings[self.function.name](self.argument.evaluate(**bindings))
    def expand(self):
        return Apply(self.function,self.argument.expand())

In [52]:
class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) * self.exp2.evaluate(**bindings)
    def expand(self):
        expanded1=self.exp1.expand()
        expanded2=self.exp2.expand()
        if isinstance(expanded1,Sum):
            return Sum(*[Product(e,expanded2).expand() for e in expanded1.exps])
        elif isinstance(expanded2,Sum):
            return Sum(*[Product(expanded1,e).expand() for e in expanded2.exps])
        else:
            return Product(expanded1,expanded2)

In [ ]:
def contains(exp,var):
    if isinstance(exp,Variable):
        return exp.symbol==var.
    elif isinstance(exp,Number):
        return 
    elif isinstance(exp,Sum):
        return 
    elif isinstance(exp,Product):
        return
    elif isinstance(exp,Power):
        return 
    elif isinstance(exp,Apply):
        return 
    else:
        raise TypeError("Not a vaild expression")